In [87]:
import json
import glob
import pandas as pd
import numpy as np
import tqdm
from sklearnex import patch_sklearn
patch_sklearn()
import sklearn
from numpy import asarray
from PIL import Image
import torch
from torchvision.transforms import ToTensor
from torch import flatten
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

data = '/Users/paddy/Desktop/AiCore/facebook_ml/final_dataset/image_data.json'
images = '/Users/paddy/Desktop/AiCore/facebook_ml/images_raw'
images_corrected = '/Users/paddy/Desktop/AiCore/facebook_ml/Images'

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
image_list = glob.glob(f'{images}/*.jpg')
df = pd.read_json(data)
# df = df.drop(['Unnamed: 0', 'bucket_link', 'image_ref', 'create_time'], axis=1)

df

,image_id,id2,bucket_link,image_ref,create_time,price,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles,id
0,912bb259-3ad9-457b-9db1-ce1da9016057,5f5f57d7-778f-4336-bb10-b43863418c8c,https://aicore-product-images.s3.amazonaws.com...,koQAAOSwhZRiE5dD,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,243809c0-9cfc-4486-ad12-3b7a16605ba9
1,b166d305-b852-4bdd-83f4-465b20da94fa,5f5f57d7-778f-4336-bb10-b43863418c8c,https://aicore-product-images.s3.amazonaws.com...,9vgAAOSw8sRiE5dE,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,dcbe214e-cb4b-4f18-ab16-218ec12ab68b
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,https://aicore-product-images.s3.amazonaws.com...,FlwAAOSwCkpiFNlV,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,6f228cd8-a2d6-4f23-858b-4a77a2f42d7c
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,https://aicore-product-images.s3.amazonaws.com...,4gIAAOSw4ORiFNlZ,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,2275f5f2-ee51-410c-8d85-e5923285a864
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf,https://aicore-product-images.s3.amazonaws.com...,sHoAAOSwQoRiD7vf,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,c0e5ef79-09c0-4cb7-9b7c-edd3d53aa530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,ced20b15-f71f-4f18-8b42-8bec96e177c6,a7594f7c-cfd9-49b4-b504-76181fbad85c,https://aicore-product-images.s3.amazonaws.com...,Q3AAAOSw4dZiGUa9,2022-02-27,55.50,0,0,0,0,0,0,0,0,0,0,0,1,a9e31ee1-c706-42a9-97d9-4ca257e4c6f3
5763,ea77b2e0-498b-4015-8884-edd7780c765a,a7594f7c-cfd9-49b4-b504-76181fbad85c,https://aicore-product-images.s3.amazonaws.com...,FWwAAOSwxKhiGUa~,2022-02-27,88.00,0,0,0,0,0,0,0,0,0,0,0,1,f3bcf410-3972-4021-bc0f-dae1892f8793
5764,0f77f8cc-e8bf-4f7d-89ea-62a92d6dbc3f,37a7232e-3579-47f3-8239-b82f516f5473,https://aicore-product-images.s3.amazonaws.com...,i~UAAOSwVj5iGUkx,2022-02-27,63.00,0,0,0,0,0,0,0,0,0,0,0,1,dd1a90ff-4a1f-4ff8-8183-381d79447bf2
5765,89103db6-f8b4-41bd-b03d-8e56a6b72fa9,37a7232e-3579-47f3-8239-b82f516f5473,https://aicore-product-images.s3.amazonaws.com...,puAAAOSw791iGUkz,2022-02-27,179.99,0,0,0,0,0,0,0,0,0,0,0,1,ebe8d613-4aa0-477c-8f1a-256ff8d4f248


In [3]:
df_for_model = df.drop(['bucket_link', 'image_ref', 'create_time','image_ref', 'id', 'id2','price', 'image_id'], axis=1)
# df_for_model = df.drop(['uuid', 'price', 'id', 'id2'], axis=1)
df_for_model

,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles
0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5762,0,0,0,0,0,0,0,0,0,0,0,1
5763,0,0,0,0,0,0,0,0,0,0,0,1
5764,0,0,0,0,0,0,0,0,0,0,0,1
5765,0,0,0,0,0,0,0,0,0,0,0,1


In [4]:
df_column_classification = {k: i for i, k in enumerate(df_for_model.columns)}
df_column_classification

{'category_Baby & Kids Stuff ': 0,
 'category_Clothes, Footwear & Accessories ': 1,
 'category_Computers & Software ': 2,
 'category_DIY Tools & Materials ': 3,
 'category_Health & Beauty ': 4,
 'category_Home & Garden ': 5,
 'category_Music, Films, Books & Games ': 6,
 'category_Office Furniture & Equipment ': 7,
 'category_Other Goods ': 8,
 'category_Phones, Mobile Phones & Telecoms ': 9,
 'category_Sports, Leisure & Travel ': 10,
 'category_Video Games & Consoles ': 11}

In [5]:

df_for_model['class'] = df_for_model[df_for_model.columns[0:]].apply(lambda x: x.idxmax(), axis=1)
df_for_model

,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles,class
0,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden
1,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden
2,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden
3,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden
4,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles
5763,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles
5764,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles
5765,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles


In [6]:
df_for_model['image_id'] =  df['image_id']
df_for_model.image_id = df_for_model.image_id.astype(str)
df_for_model

,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles,class,image_id
0,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,912bb259-3ad9-457b-9db1-ce1da9016057
1,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,b166d305-b852-4bdd-83f4-465b20da94fa
2,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,68f5a29d-0075-4d60-81c1-ab684a82e50c
3,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,f6a309d7-d247-446a-9b5e-aceefdd4334d
4,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,2c2b3a6f-15b3-4289-937a-15482d9f5781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles,ced20b15-f71f-4f18-8b42-8bec96e177c6
5763,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles,ea77b2e0-498b-4015-8884-edd7780c765a
5764,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles,0f77f8cc-e8bf-4f7d-89ea-62a92d6dbc3f
5765,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles,89103db6-f8b4-41bd-b03d-8e56a6b72fa9


In [7]:
df_for_model['classification_number'] = df_for_model['class'].map(df_column_classification)
df_for_model

,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles,class,image_id,classification_number
0,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,912bb259-3ad9-457b-9db1-ce1da9016057,5
1,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,b166d305-b852-4bdd-83f4-465b20da94fa,5
2,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,68f5a29d-0075-4d60-81c1-ab684a82e50c,5
3,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,f6a309d7-d247-446a-9b5e-aceefdd4334d,5
4,0,0,0,0,0,1,0,0,0,0,0,0,category_Home & Garden,2c2b3a6f-15b3-4289-937a-15482d9f5781,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles,ced20b15-f71f-4f18-8b42-8bec96e177c6,11
5763,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles,ea77b2e0-498b-4015-8884-edd7780c765a,11
5764,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles,0f77f8cc-e8bf-4f7d-89ea-62a92d6dbc3f,11
5765,0,0,0,0,0,0,0,0,0,0,0,1,category_Video Games & Consoles,89103db6-f8b4-41bd-b03d-8e56a6b72fa9,11


In [28]:
list_model = list(zip(df_for_model.image_id, df_for_model.classification_number))
print(len(list_model))
print(list_model[0])


5767
('912bb259-3ad9-457b-9db1-ce1da9016057', 5)


In [90]:
image_id = list_model
# image_list = 


def image_correct_prop():
        resized_img = 128
        for img in range(len(image_id)):
            image_ite = Image.open(f'{images}/{image_id[img][0]}.jpg')
            background = Image.new(mode='RGB', size=(resized_img, resized_img)) 
            original_img = image_ite.size
            max_dim = max(image_ite.size)
            ratio = resized_img / max_dim
            img_corr_ratio =(int(original_img[0] * ratio), int(original_img[1] * ratio))
            img_corr = image_ite.resize(img_corr_ratio)
            # print(img_corr)
            background.paste(img_corr, (((resized_img - img_corr_ratio[0]) // 2), ((resized_img - img_corr_ratio[1]) // 2)))
            # print(background)
            background.save(f'{images_corrected}/{image_id[img][0]}.jpg')
image_correct_prop()

In [26]:
image_id = list_model
image_array = []

def image_to_tensor():
        for img in image_id:
            image = Image.open(f'{images_corrected}/{img[0]}.jpg')
            print(image)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            flatten_numpy = flattened.numpy()
            # print(flattened.dtype)
            image_array.append(flatten_numpy)
            # index_of_image = image_id.values()
            # image_tuple = ((image_array, index_of_image))
            # image_array.append(image_tuple)
            # print(empty)
            print(flatten_numpy)
        return image_array
image_to_tensor()
print(len(image_array))





<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0DA5D15E0>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0C0661C10>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0C0661910>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0C0661C10>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0C0661910>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0D580A3A0>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0C0661910>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0D792C340>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7FB0C0661C10>
[0. 0. 0. ... 0. 0. 0.]
<PIL.JpegImagePlugin.JpegImageFile image mode=

In [48]:
list_of_tuples = []
print(len(image_array))
print(len(list_model))
print(list_model[0][0])
def joiner():
    for i in range(len(image_array)):
        image_tuple = image_array[i], list_model[i][1]
        list_of_tuples.append(image_tuple)
        print(image_tuple)
    return
joiner()
print(list_of_tuples)

5767
5767
912bb259-3ad9-457b-9db1-ce1da9016057
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0.02745098, 0.7882353 , 0.7764706 , ..., 0.76862746, 0.78431374,
       0.01568628], dtype=float32), 5)
(array([0.00392157, 0.00392157, 0.00392157, ..., 0.00784314, 0.00784314,
       0.        ], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., ..., 0., 0., 0.], dtype=f

In [59]:
print(list_of_tuples[5500])

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 10)


In [ ]:
class image_setup:
    def __init__(self, df: pd):
        self.df = df
        self.image_list_fixed = glob.glob(f'{images_corrected}/*.jpg')
        self.image_id_list = []
        self.class_list = []
        self.flattened = []
        self.list_of_tuples = []
        self.df_cleaned(df)
        self.classification_names_list(df)
        self.image_correct_prop()
        self.image_to_tensor()
        self.training_tuples()
        self.model_setup()

    
    def df_cleaned(self, df: pd):
        self.df = self.df.drop(['Unnamed: 0', 'bucket_link', 'image_ref', 'create_time'], axis=1)
        self.df = self.df.drop(['uuid', 'price', 'id.1'], axis=1)
        column_indicies = [1,2,3,4,5,6,7,8,9,10,11,12]
        classification_columns = ['0','1','2','3','4','5','6','7','8','9','10','11']
        old_names = self.df.columns[column_indicies]
        self.df.rename(columns=dict(zip(old_names, classification_columns)),inplace=True)
        self.df = self.df.drop(['id'], axis=1)


    def classification_names_list(self, df: pd):
        self.df['new'] = self.df.apply(lambda x: x.index[x == 1].tolist(), axis=1)
        flat_list = []
        class_list = self.df['new'].tolist()
        for sublist in class_list:
            for item in sublist:
                flat_list.append(item)
            self.class_list = list(map(int, flat_list))


    def image_correct_prop(self):
        resized_img = 128
        for n, img in enumerate(self.image_list_fixed):
            image_ite = Image.open(img)
            background = Image.new(mode='RGB', size=(resized_img, resized_img)) 
            original_img = image_ite.size
            max_dim = max(image_ite.size)
            ratio = resized_img / max_dim
            img_corr_ratio =(int(original_img[0] * ratio), int(original_img[1] * ratio))
            img_corr = image_ite.resize(img_corr_ratio)
            # print(img_corr)
            background.paste(img_corr, (((resized_img - img_corr_ratio[0]) // 2), ((resized_img - img_corr_ratio[1]) // 2)))
            # print(background)
            background.save(f'{images_corrected}/{img[50:]}')

    
    def image_to_tensor(self):
        for i, img in enumerate(self.image_list_fixed):
            image = Image.open(img)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            flatten_numpy = flattened.numpy()
            # print(self.flattened)
            self.flattened.append(flatten_numpy)

        return self.flattened

    def training_tuples(self):
        for i in range(len(self.class_list)):
                # print(image_array[i])
                # print(image_array[i])
                ready_tuple = (self.flattened[i], self.class_list[i])
                self.list_of_tuples.append(ready_tuple)
                # print(self.list_of_tuples)
        return self.list_of_tuples

    def model_setup(self):
        m = len(self.list_of_tuples)
        # sh = image_array[0].shape
        array_size = 49152
        # list_of_tuples
        # print(len(self.list_of_tuples))
        X = np.zeros((m, array_size))
        y = np.zeros(m)
        # print(X.shape, y.shape)
        for item in range(m):
            features, labels = self.list_of_tuples[item]
            # print(features.shape)
            X[item, :] = features
            y[item] = labels
            # print(X)
            # print(y)
        model = LogisticRegression(max_iter= 100)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=59)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        print ('acc:', accuracy_score(y_test, pred))
        print('prec:', classification_report(y_test, pred))
            
    


            
    
    pass
image_setup(df)

In [ ]:
print(image_array[1].shape)
size = 128*128*3
n = len(image_array)
print(len(image_array))
print(image_array[1])


In [70]:
test_tuple = list_of_tuples[-500:]
print(test_tuple)

[(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0.        , 0.00784314, 0.        , ..., 0.        , 0.00392157,
       0.        ], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 1), (array([0., 0., 

In [71]:
# m = no of labels
#  features = 49152 = items in list of flattened image :)
m = len(test_tuple)
sh = image_array[0].shape
array_size = 49152
list_of_tuples
print(len(list_of_tuples))
X = np.zeros((m, array_size))
y = np.zeros(m)
print(X.shape, y.shape)
for item in range(m):
    features, labels = test_tuple[item]
    # print(features.shape)
    X[item, :] = features
    y[item] = labels
    # print(X)
    print(y)

5767
(500, 49152) (500,)
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0

In [72]:
model = LogisticRegression(max_iter= 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print ('acc:', accuracy_score(y_test, pred))
print('prec:', classification_report(y_test, pred))

acc: 0.24
prec:               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         6
         1.0       0.38      0.39      0.39        33
         2.0       0.14      0.08      0.11        12
         3.0       0.07      0.10      0.08        10
         4.0       0.00      0.00      0.00         2
         6.0       0.00      0.00      0.00         2
         7.0       0.00      0.00      0.00         1
         8.0       0.12      0.14      0.13         7
         9.0       0.17      0.10      0.12        10
        10.0       0.35      0.46      0.40        13
        11.0       0.33      0.25      0.29         4

    accuracy                           0.24       100
   macro avg       0.14      0.14      0.14       100
weighted avg       0.24      0.24      0.23       100



/Users/paddy/opt/anaconda3/lib/python3.8/site-packages/daal4py/sklearn/linear_model/logistic_path.py:459: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
